## Propuesta Interfáz
Aqui hay una propuesta interfáz gráfica para que el usuario decida sus operaciones, por el momento se ha llamado solo a la clase aritmética. 


In [ ]:
import tkinter as tk
from tkinter import messagebox
from modules.arithmetic import Aritmetica

class CalculatorApp:
    def __init__(self, root):
        self.calc = Aritmetica()
        self.root = root
        self.root.title("Calculadora Clásica")
        self.root.geometry("400x550")
        self.root.resizable(False, False)
        
        # Variables de estado
        self.current_input = ""
        self.previous_input = ""
        self.operation = None
        self.reset_screen = False
        
        self.create_widgets()
    
    def create_widgets(self):
        # Frame principal
        main_frame = tk.Frame(self.root, padx=10, pady=10, bg='#2c3e50')
        main_frame.pack(expand=True, fill='both')
        
        # PANTALLA (Display)
        screen_frame = tk.Frame(main_frame, bg='#34495e', relief='sunken', bd=3)
        screen_frame.pack(pady=10, fill='x')
        
        self.display = tk.Entry(screen_frame, font=("Arial", 24), 
                               justify='right', bd=0, bg='#ecf0f1', 
                               fg='#2c3e50', readonlybackground='#ecf0f1')
        self.display.pack(pady=15, padx=10, fill='x')
        self.display.insert(0, "0")
        self.display.config(state='readonly')
        
        # TECLADO
        keyboard_frame = tk.Frame(main_frame, bg='#2c3e50')
        keyboard_frame.pack(expand=True, fill='both')
        
        # Botones en grid 5x4
        buttons = [
            # Fila 1
            ('C', self.clear_all, '#e74c3c', 1, 0),
            ('⌫', self.delete_last, '#e67e22', 1, 1),
            ('±', self.negate, '#3498db', 1, 2),
            ('÷', lambda: self.set_operation('÷'), '#2980b9', 1, 3),
            
            # Fila 2
            ('7', lambda: self.add_number('7'), '#34495e', 2, 0),
            ('8', lambda: self.add_number('8'), '#34495e', 2, 1),
            ('9', lambda: self.add_number('9'), '#34495e', 2, 2),
            ('×', lambda: self.set_operation('×'), '#2980b9', 2, 3),
            
            # Fila 3
            ('4', lambda: self.add_number('4'), '#34495e', 3, 0),
            ('5', lambda: self.add_number('5'), '#34495e', 3, 1),
            ('6', lambda: self.add_number('6'), '#34495e', 3, 2),
            ('-', lambda: self.set_operation('-'), '#2980b9', 3, 3),
            
            # Fila 4
            ('1', lambda: self.add_number('1'), '#34495e', 4, 0),
            ('2', lambda: self.add_number('2'), '#34495e', 4, 1),
            ('3', lambda: self.add_number('3'), '#34495e', 4, 2),
            ('+', lambda: self.set_operation('+'), '#2980b9', 4, 3),
            
            # Fila 5
            ('0', lambda: self.add_number('0'), '#34495e', 5, 0),
            ('.', self.add_decimal, '#34495e', 5, 1),
            ('=', self.calculate, '#27ae60', 5, 2),
            ('📊', self.show_last_result, '#8e44ad', 5, 3)
        ]
        
        # Configurar grid
        for i in range(4):
            keyboard_frame.grid_columnconfigure(i, weight=1)
        for i in range(6):
            keyboard_frame.grid_rowconfigure(i, weight=1)
        
        # Crear botones
        for (text, command, color, row, col) in buttons:
            btn = tk.Button(keyboard_frame, text=text, command=command,
                           font=("Arial", 16, "bold"), bg=color, fg='white',
                           relief='raised', bd=3, height=2)
            btn.grid(row=row, column=col, sticky='nsew', padx=2, pady=2)
        
        # Estado
        self.status_label = tk.Label(main_frame, text="Listo", 
                                    font=("Arial", 10), fg='#ecf0f1', bg='#2c3e50')
        self.status_label.pack(pady=5)
    
    def update_display(self):
        """Actualiza la pantalla con el input actual"""
        self.display.config(state='normal')
        self.display.delete(0, tk.END)
        self.display.insert(0, self.current_input or "0")
        self.display.config(state='readonly')
    
    def add_number(self, number):
        """Agrega un número al input actual"""
        if self.reset_screen:
            self.current_input = ""
            self.reset_screen = False
        
        if self.current_input == "0":
            self.current_input = number
        else:
            self.current_input += number
        
        self.update_display()
        self.status_label.config(text=f"Input: {self.current_input}")
    
    def add_decimal(self):
        """Agrega punto decimal"""
        if self.reset_screen:
            self.current_input = ""
            self.reset_screen = False
        
        if '.' not in self.current_input:
            if not self.current_input:
                self.current_input = "0."
            else:
                self.current_input += '.'
            self.update_display()
    
    def delete_last(self):
        """Elimina el último carácter"""
        if self.current_input:
            self.current_input = self.current_input[:-1]
            if not self.current_input:
                self.current_input = "0"
            self.update_display()
    
    def clear_all(self):
        """Limpia toda la calculadora"""
        self.current_input = ""
        self.previous_input = ""
        self.operation = None
        self.reset_screen = False
        self.update_display()
        self.status_label.config(text="Calculadora reiniciada")
    
    def negate(self):
        """Cambia el signo del número actual"""
        if self.current_input and self.current_input != "0":
            if self.current_input[0] == '-':
                self.current_input = self.current_input[1:]
            else:
                self.current_input = '-' + self.current_input
            self.update_display()
    
    def set_operation(self, op):
        """Establece la operación a realizar"""
        if self.current_input:
            self.previous_input = self.current_input
            self.current_input = ""
            self.operation = op
            self.reset_screen = True
            self.status_label.config(text=f"Operación: {op}")
    
    def calculate(self):
        """Realiza el cálculo"""
        if not self.operation or not self.previous_input or not self.current_input:
            return
        
        try:
            num1 = float(self.previous_input)
            num2 = float(self.current_input)
            
            # Realizar operación según el símbolo
            if self.operation == '+':
                resultado = self.calc.sumar(num1, num2)
            elif self.operation == '-':
                resultado = self.calc.restar(num1, num2)
            elif self.operation == '×':
                resultado = self.calc.multiplicar(num1, num2)
            elif self.operation == '÷':
                if num2 == 0:
                    raise ZeroDivisionError("División por cero")
                resultado = self.calc.dividir(num1, num2)
            
            # Mostrar resultado
            self.current_input = str(resultado)
            self.previous_input = ""
            self.operation = None
            self.reset_screen = True
            self.update_display()
            self.status_label.config(text=f"Resultado: {resultado}")
            
        except ZeroDivisionError:
            messagebox.showerror("Error", "No se puede dividir por cero")
            self.clear_all()
        except ValueError:
            messagebox.showerror("Error", "Valores inválidos")
            self.clear_all()
        except Exception as e:
            messagebox.showerror("Error", f"Error inesperado: {str(e)}")
            self.clear_all()
    
    def show_last_result(self):
        """Muestra el último resultado calculado"""
        ultimo = self.calc.ultimo_resultado
        messagebox.showinfo("Último Resultado", f"El último resultado fue: {ultimo}")

# Función principal
def main():
    root = tk.Tk()
    app = CalculatorApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()